In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import warnings
import logging
from neuralprophet import NeuralProphet, set_log_level
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm

warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger('neuralprophet').setLevel(logging.WARNING)  # Suppress logs below WARNING

# Disable logging messages unless there is an error
set_log_level("ERROR")

Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


In [8]:
# Load data files
data = pd.read_csv("C:/Data/M5store_2.csv")

# Ensure proper formatting
data['ds'] = pd.to_datetime(data['d'])
data = data.sort_values(by=['store_id', 'ds']).reset_index(drop=True)

data.rename(columns={'revenue': 'y'}, inplace=True)

In [23]:
# Create pivot table
pivot_data = data.pivot(index='ds', columns='store_id', values='y')
pivot_data.columns = [f'sales_store_{store}' for store in pivot_data.columns]

# Merge the pivoted data with the original dataset
data = pd.merge(
    data, 
    pivot_data.reset_index(), 
    on=['ds'], 
    how='left'
)

# Drop self-sales for each store
for store in data['store_id'].unique():
    feature_to_drop = f'sales_store_{store}'
    data.loc[data['store_id'] == store, feature_to_drop] = None

# Create lagged values for all columns starting with 'sales_store_'
#for col in [col for col in data.columns if col.startswith('sales_store_')]:
#    # Replace the column with its lagged version
#    data[col] = data.groupby('store_id')[col].shift(1)  # Replace '1' with desired lag period

In [9]:
# Calculate total demand across all stores for each date
total_demand = data.groupby('ds')['y'].sum().reset_index()
total_demand.rename(columns={'y': 'total_sales_all_stores'}, inplace=True)

# Merge total demand back into the original data
data = pd.merge(data, total_demand, on='ds', how='left')

# Subtract the current store's sales to get demand from other stores
data['demand_other_stores'] = data['total_sales_all_stores'] - data['y']

# Drop the intermediate column if not needed
data.drop(columns=['total_sales_all_stores'], inplace=True)

In [10]:
data.drop(columns=['index'], inplace=True)
data.drop(columns=['d'], inplace=True)

In [11]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19410 entries, 0 to 19409
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   store_id             19410 non-null  object        
 1   y                    19410 non-null  float64       
 2   sell_price           19410 non-null  float64       
 3   ds                   19410 non-null  datetime64[ns]
 4   demand_other_stores  19410 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 758.3+ KB
None


In [18]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
from tqdm import tqdm

def calculate_mase(y_true, y_pred, y_train):
    """
    Calculate the Mean Absolute Scaled Error (MASE).
    """
    mae_forecast = np.mean(np.abs(y_true - y_pred))
    mae_naive = np.mean(np.abs(y_train[1:] - y_train[:-1]))
    return mae_forecast / mae_naive

# Initialize results
mase_scores = []
store_results = {}

# Parameters
m = 28  # Number of observations to leave out for validation
h = 14   # Forecast horizon
n = m - h  # Number of rolling forecasts to generate

# Iterate over each store
for store in tqdm(data['store_id'].unique(), desc="Processing stores"):
    store_data = data[data['store_id'] == store].copy()
    store_data = store_data.sort_values('ds').reset_index(drop=True)

    model = NeuralProphet(
        n_lags=7,
        n_forecasts=h,
        ar_layers=[16, 16],
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        n_changepoints=10,
        changepoints_range=0.8)
    model.add_country_holidays("US")
    model.add_future_regressor("sell_price")
    model.add_lagged_regressor("demand_other_stores", n_lags=1)
    
    # Split data into training and testing sets
    train_data = store_data.iloc[:-m]
    #test_data = store_data.iloc[-m:]
    #test_data = test_data.reset_index(drop=True)
    
    # Train the model
    model.fit(train_data[['ds', 'y', 'sell_price', 'demand_other_stores']], freq="D", minimal=True, epochs=120)
    
    # Rolling forecast
    y_preds = []
    y_trues = []
    
    for i in range(n):
        forecast_data = store_data.iloc[:-m + i + h].copy()

        # Make forecast
        forecast = model.predict(forecast_data[['ds', 'y', 'sell_price', 'demand_other_stores']])
        y_pred = forecast.iloc[-1][f"yhat{h}"]
        y_true = forecast.iloc[-1]['y']
        
        # Store true and predicted values
        y_preds.append(y_pred)
        y_trues.append(y_true)
        #y_trues.append(test_data.iloc[h + i - 1]['y'])
    
    # Calculate MASE for the store
    mase = calculate_mase(np.array(y_trues), np.array(y_preds), train_data['y'].values)
    store_results[store] = mase
    mase_scores.append(mase)

# Calculate the average MASE across all stores
average_mase = np.mean(mase_scores)

# Create a DataFrame for the results
store_results_df = pd.DataFrame.from_dict(store_results, orient='index', columns=['MASE'])
store_results_df.loc['Average'] = average_mase

# Display the results
print(store_results_df)

Processing stores:   0%|                                                                        | 0/10 [00:00<?, ?it/s]

Finding best initial lr:   0%|          | 0/232 [00:00<?, ?it/s]

Processing stores:  10%|██████▍                                                         | 1/10 [00:39<05:59, 39.98s/it]

Finding best initial lr:   0%|          | 0/232 [00:00<?, ?it/s]

Processing stores:  20%|████████████▊                                                   | 2/10 [01:17<05:09, 38.74s/it]

...

Finding best initial lr:   0%|          | 0/232 [00:00<?, ?it/s]

Processing stores:  90%|█████████████████████████████████████████████████████████▌      | 9/10 [05:44<00:38, 38.07s/it]

Finding best initial lr:   0%|          | 0/232 [00:00<?, ?it/s]

Processing stores: 100%|███████████████████████████████████████████████████████████████| 10/10 [06:22<00:00, 38.21s/it]

             MASE
CA_1     0.584234
CA_2     0.877810
CA_3     0.708761
CA_4     0.793601
TX_1     0.563215
TX_2     0.324389
TX_3     0.812571
WI_1     0.653963
WI_2     1.028290
WI_3     1.107245
Average  0.745408
